In [1]:
!pip install scikit-plot

In [2]:
!pip install dask-ml

In [3]:
!pip install imbalanced-learn

In [4]:
!pip install fastparquet

In [1]:
import time
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import sys 
#Sklearn
from sklearn.metrics import *
import scikitplot as skplt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
#Imbalanced learn
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, ADASYN
from imblearn.pipeline import Pipeline, make_pipeline
#Dask et Parquet
import dask.dataframe as dd 
import fastparquet
from dask_ml.preprocessing import DummyEncoder
import pickle
import dask_ml 
from dask import delayed
from dask_ml.preprocessing import StandardScaler
#One class SVM
from sklearn.kernel_approximation import Nystroem
from sklearn.ensemble import IsolationForest
from sklearn import linear_model
from sklearn import svm

In [2]:
os.chdir('C:\\Users\\cornuch\\Desktop\\SISE\\Fouille_donnees_massives\\projet_fouille')

# 1. Importation des fichiers et préparation

## 1.1 Importation

In [3]:
df_train=dd.read_parquet('train_fast')
df_train.head()

,FlagImpaye,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
0,0,20.00,0,551,0.000000,0.000000,0.000000,52.076034,0,1,4.0,0.00,7
1,0,20.00,0,551,0.000000,0.000000,0.000000,52.076034,1,2,4.0,8.61,7
2,0,57.64,0,549,0.000000,0.000000,0.000000,52.076034,0,1,4.0,0.00,7
3,0,54.29,1,267,0.000000,0.000000,0.000000,53.554234,0,1,4.0,0.00,7
4,0,26.90,0,549,0.003769,8.586333,0.001192,52.076034,1,1,4.0,32.25,8


In [4]:
df_test=dd.read_parquet('test_fast')
df_test.head()

,FlagImpaye,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
3899362,0,60.00,0,302,0.000000,0.000000,0.000000,52.076034,0,1,4.000000,0.00,7
3899363,0,25.00,0,547,0.000000,0.000000,0.000000,52.076034,0,1,4.000000,0.00,8
3899364,0,32.00,0,152,0.000000,0.000000,0.000000,52.076034,0,1,4.000000,0.00,8
3899365,0,20.88,0,393,0.007590,0.182221,0.002092,34.118277,4,1,1.539155,71.51,8
3899366,0,20.21,0,152,0.018994,-1.024399,0.005682,52.076034,0,1,4.000000,0.00,8


In [5]:
# Définir la variable cible 
y_train = df_train["FlagImpaye"]
y_test = df_test["FlagImpaye"]

In [6]:
# Définir les variables explicatives 
X_train = df_train.drop(["FlagImpaye"], axis = "columns")
X_test = df_test.drop(["FlagImpaye"], axis = "columns")

**Importation des fichiers équilibrés avec BorderlineSMOTE**

In [84]:
df_train_borderS=dd.read_parquet('train_borderS')
# Définir la variable cible 
y_train_border = df_train_borderS["FlagImpaye"]
# Définir les variables explicatives 
X_train_border = df_train_borderS.drop(["FlagImpaye"], axis = "columns")
X_train_border.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
0,20.00,0,551,0.000000,0.000000,0.000000,52.076034,0,1,4.0,0.00,7
1,20.00,0,551,0.000000,0.000000,0.000000,52.076034,1,2,4.0,8.61,7
2,57.64,0,549,0.000000,0.000000,0.000000,52.076034,0,1,4.0,0.00,7
3,54.29,1,267,0.000000,0.000000,0.000000,53.554234,0,1,4.0,0.00,7
4,26.90,0,549,0.003769,8.586333,0.001192,52.076034,1,1,4.0,32.25,8


In [92]:
df_train_ada=dd.read_parquet('train_ADA')
# Définir la variable cible 
y_train_ada = df_train_ada["FlagImpaye"]
# Définir les variables explicatives 
X_train_ada = df_train_ada.drop(["FlagImpaye"], axis = "columns")
X_train_ada.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
0,20.00,0,551,0.000000,0.000000,0.000000,52.076034,0,1,4.0,0.00,7
1,20.00,0,551,0.000000,0.000000,0.000000,52.076034,1,2,4.0,8.61,7
2,57.64,0,549,0.000000,0.000000,0.000000,52.076034,0,1,4.0,0.00,7
3,54.29,1,267,0.000000,0.000000,0.000000,53.554234,0,1,4.0,0.00,7
4,26.90,0,549,0.003769,8.586333,0.001192,52.076034,1,1,4.0,32.25,8


## 1.2 Centrage et réduction

In [8]:
# Instanciation de StandardScaler pour centrer-réduire les données
scaler = StandardScaler()

In [9]:
# Centrer-réduire les dataframe d'apprentissage
X_train_scale = scaler.fit_transform(X_train)
X_train_scale.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
0,-0.420302,-0.581824,1.017656,-0.084590,-0.239505,-0.275801,0.818043,-0.043817,-0.194165,-0.451325,-0.202861,-2.122399
1,-0.420302,-0.581824,1.017656,-0.084590,-0.239505,-0.275801,0.818043,-0.043812,4.975724,-0.451325,0.038808,-2.122399
2,-0.022127,-0.581824,1.008573,-0.084590,-0.239505,-0.275801,0.818043,-0.043817,-0.194165,-0.451325,-0.202861,-2.122399
3,-0.057565,1.545332,-0.272152,-0.084590,-0.239505,-0.275801,0.906727,-0.043817,-0.194165,-0.451325,-0.202861,-2.122399
4,-0.347310,-0.581824,1.008573,-0.084434,0.637044,-0.273204,0.818043,-0.043812,-0.194165,-0.451325,0.702346,-1.820898


In [10]:
# Centrer-réduire les dataframe de test
X_test_scale = scaler.fit_transform(X_test)
X_test_scale.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
3899362,-0.016999,0.0,0.265719,-0.080269,-0.043960,-0.317899,0.774799,-0.036921,-0.18432,-0.568544,-0.132257,-2.235916
3899363,-0.277531,0.0,1.427785,-0.080269,-0.043960,-0.317899,0.774799,-0.036921,-0.18432,-0.568544,-0.132257,-1.929154
3899364,-0.225425,0.0,-0.445750,-0.080269,-0.043960,-0.317899,0.774799,-0.036921,-0.18432,-0.568544,-0.132257,-1.929154
3899365,-0.308200,0.0,0.697344,-0.080074,-0.023966,-0.314458,-0.277090,-0.036897,-0.18432,-5.906429,1.259186,-1.929154
3899366,-0.313187,0.0,-0.445750,-0.079781,-0.156360,-0.308551,0.774799,-0.036921,-0.18432,-0.568544,-0.132257,-1.929154


In [85]:
X_train_border_scale = scaler.fit_transform(X_train_border)
X_train_border_scale.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
0,-0.515961,-0.647657,1.295485,-0.124668,0.112202,-0.264384,0.628654,-0.049253,-0.196192,-0.435922,-0.204627,-2.301345
1,-0.515961,-0.647657,1.295485,-0.124668,0.112202,-0.264384,0.628654,-0.049248,4.558503,-0.435922,-0.110379,-2.301345
2,-0.175906,-0.647657,1.286605,-0.124668,0.112202,-0.264384,0.628654,-0.049253,-0.196192,-0.435922,-0.204627,-2.301345
3,-0.206172,1.124615,0.034585,-0.124668,0.112202,-0.264384,0.717859,-0.049253,-0.196192,-0.435922,-0.204627,-2.301345
4,-0.453624,-0.647657,1.286605,-0.124593,0.904495,-0.263172,0.628654,-0.049248,-0.196192,-0.435922,0.148391,-1.984147


In [107]:
X_train_ada_scale = scaler.fit_transform(X_train_ada)
X_train_ada_scale.head()

,Montant,VerifianceCPT1,D2CB,ScoringFP1,ScoringFP2,ScoringFP3,TauxImpNB_CPM,EcartNumCheq,NbrMagasin3J,DiffDateTr3,CA3TR,Heure_num
0,-0.523172,-0.631261,1.256956,-0.113913,0.080205,-0.265926,0.639664,-0.045952,-0.181309,-0.453426,-0.200586,-2.305235
1,-0.523172,-0.631261,1.256956,-0.113913,0.080205,-0.265926,0.639664,-0.045947,5.111903,-0.453426,-0.080160,-2.305235
2,-0.158515,-0.631261,1.248056,-0.113913,0.080205,-0.265926,0.639664,-0.045952,-0.181309,-0.453426,-0.200586,-2.305235
3,-0.190970,1.231665,-0.006955,-0.113913,0.080205,-0.265926,0.729507,-0.045952,-0.181309,-0.453426,-0.200586,-2.305235
4,-0.456325,-0.631261,1.248056,-0.113830,0.891926,-0.264560,0.639664,-0.045947,-0.181309,-0.453426,0.250487,-1.986605


# 2. Mesures de performances

**Maximisation de la marge**

In [15]:
def perte(Montant):
    if Montant<=20:
        Montant=Montant*0
    elif 20<Montant<=50:
        Montant=0.2*Montant
    elif 50<Montant<=100:
        Montant=0.3*Montant
    elif 100<Montant<=200:
        Montant=0.5*Montant
    elif Montant>200:
        Montant=0.8*Montant
    return(Montant)

In [16]:
def Calcul_Marge(Montant, yReel, yPred):
    # Création de dfmerge
    dfmerge = pd.concat([Montant, yReel], axis=1)
    dfmerge["Ypred"] = yPred
    
    # Création de la variable Marge
    # Création d'une colonne Marge égale au Montant
    dfmerge["Marge"] = dfmerge["Montant"]
    #Calcul du gain ou de la perte selon la différence entre réel et prédiction
    dfmerge.loc[((dfmerge["FlagImpaye"] == 1) & (dfmerge["Ypred"] == 1)), "Marge"] = 0
    dfmerge.loc[((dfmerge["FlagImpaye"] == 0) & (dfmerge["Ypred"] == 1)), "Marge"] = 0.7 * 0.05 * dfmerge["Montant"]
    dfmerge.loc[((dfmerge["FlagImpaye"] == 1) & (dfmerge["Ypred"] == 0)), "Marge"] = dfmerge["Montant"].apply(lambda x: -perte(x))
    dfmerge.loc[((dfmerge["FlagImpaye"] == 0) & (dfmerge["Ypred"] == 0)), "Marge"] = 0.05 * dfmerge["Montant"]
    
    # Calcul du Marge_totale
    Marge_totale = dfmerge["Marge"].sum()
    
    return Marge_totale

In [18]:
# On recode en -1 et 1 pour les besoins de ces algorithmes qui affectent un score de -1 ou 1
y_train_special = y_train.replace({1: -1, 0 : 1})
y_test_special = y_test.replace({1: -1, 0 : 1})

In [19]:
def resultats(predicted):
    conf_mat = delayed(confusion_matrix(y_true=y_test_special, y_pred=predicted)).compute()
    print('Confusion matrix:\n', conf_mat)
    f1 = delayed(f1_score)(y_test_special, predicted, pos_label = -1, average= 'binary').compute()
    print('F1_score :\n', f1)
    cr=delayed(classification_report(y_test_special,y_pred=predicted)).compute()
    print('Estimateurs :\n', cr)
    auc = round(roc_auc_score(y_test_special, predicted),4)
    print('Auc Score :\n', auc)
    Marge_totale = Calcul_Marge(Montant = X_test["Montant"].compute(), yReel = y_test_special.compute(), yPred=predicted)
    print("Marge = " + str(round(Marge_totale, 2)) + " euros")

# 3. Algorithmes testés

**Dans tout ce qui suit, on prendra nu=0.0065 qui correspond à la répartition des chèques impayés ou faux chèque dans nos données d'entrée**

## 3.1 One Class SVM avec noyau gaussien, polynomial, sigmoid

**Cas sans stratégie d'échantillonnage et avec paramètres "de base"**

In [20]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.0065, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform1 = Nystroem(kernel = 'rbf',random_state = 0)

clf1 = make_pipeline(transform1,model1)

# Entraînement
classifier1 = delayed(clf1.fit)(X_train_scale)

# Prédiction
y_pred1 = delayed(classifier1.predict)(X_test_scale).compute()

CPU times: total: 55.5 s
Wall time: 1min 2s


In [21]:
%%time
resultats(y_pred1)

Confusion matrix:
 [[   321   6252]
 [  4560 736277]]
F1_score :
 0.05605028810895757
Estimateurs :
               precision    recall  f1-score   support

          -1       0.07      0.05      0.06      6573
           1       0.99      0.99      0.99    740837

    accuracy                           0.99    747410
   macro avg       0.53      0.52      0.52    747410
weighted avg       0.98      0.99      0.98    747410

Auc Score :
 0.5213
Marge = 2892747.11 euros
CPU times: total: 4.25 s
Wall time: 4.16 s


In [24]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.0065, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform2 = Nystroem(kernel = 'poly',random_state = 0,degree=5)

clf2 = make_pipeline(transform2,model1)

# Entraînement
classifier2 = delayed(clf2.fit)(X_train_scale)

# Prédiction
y_pred2 = delayed(classifier2.predict)(X_test_scale).compute()

CPU times: total: 1min 43s
Wall time: 46.6 s


In [25]:
%%time
resultats(y_pred2)

Confusion matrix:
 [[  2330   4243]
 [317002 423835]]
F1_score :
 0.014298645310750067
Estimateurs :
               precision    recall  f1-score   support

          -1       0.01      0.35      0.01      6573
           1       0.99      0.57      0.73    740837

    accuracy                           0.57    747410
   macro avg       0.50      0.46      0.37    747410
weighted avg       0.98      0.57      0.72    747410

Auc Score :
 0.4633
Marge = 19053786.29 euros
CPU times: total: 8.97 s
Wall time: 4.44 s


In [26]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.0065, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform3 = Nystroem(kernel = 'sigmoid',random_state = 0)

clf3 = make_pipeline(transform3,model1)

# Entraînement
classifier3 = delayed(clf3.fit)(X_train_scale)

# Prédiction
y_pred3 = delayed(classifier2.predict)(X_test_scale).compute()

CPU times: total: 1min 47s
Wall time: 49.1 s


In [27]:
%%time
resultats(y_pred3)

Confusion matrix:
 [[  2330   4243]
 [317002 423835]]
F1_score :
 0.014298645310750067
Estimateurs :
               precision    recall  f1-score   support

          -1       0.01      0.35      0.01      6573
           1       0.99      0.57      0.73    740837

    accuracy                           0.57    747410
   macro avg       0.50      0.46      0.37    747410
weighted avg       0.98      0.57      0.72    747410

Auc Score :
 0.4633
Marge = 19053786.29 euros
CPU times: total: 9.02 s
Wall time: 4.4 s


## 3.2 Modification des paramètres pour chaque noyau

In [37]:
def Search_OneClassSVM(X_train_scale, X_test_scale, y_test_special, kernel, degree = None):    
    
    # Hyperparamètres
    nu = [0.0065, 0.009, 0.01]
    gamma = [10, 2, 0.01, 0.001]
    n_components = [5, 15, 25]
    
    # Instanciation pour le stockage des résultats de performance
    results_predictions = []

    # Tests de toutes les combinaisons
    for i in nu:
        for j in gamma:
            for l in n_components:
                # Instanciation du modèle avec la kernel approximation     
                model = linear_model.SGDOneClassSVM(nu=i,random_state=0, tol=1e-7, max_iter = 5000)
                transform = Nystroem(kernel = kernel, gamma=j, n_components = l, random_state=0, degree = degree, n_jobs = -2)

                # Pipeline
                clf = make_pipeline(transform, model)

                # Entraînement
                classifier = delayed(clf.fit)(X_train_scale)

                # Prédiction
                predicted = delayed(classifier.predict)(X_test_scale).compute()
                

                # Stockage des résultats de performance
                results_predictions.append({'f1-score' : delayed(f1_score)(y_test_special, predicted, pos_label = -1, average= 'binary').compute(),
                                     'auc' : round(roc_auc_score(y_test_special, predicted),4),
                                     'nu' : i,
                                     'gamma' : j,
                                     'n_components' : l
                                   })

    # Conversion en DataFrame
    results_predictions = pd.DataFrame(results_predictions)
    results_predictions = results_predictions.sort_values(by='auc', ascending=False)
    return results_predictions

In [38]:
%%time 
Results_rbf = Search_OneClassSVM(X_train_scale, X_test_scale, y_test_special, kernel = 'rbf')

CPU times: total: 22min 58s
Wall time: 7min 26s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [39]:
Results_rbf.head(10)

,f1-score,auc,nu,gamma,n_components
33,0.063436,0.5272,0.01,0.001,5
30,0.063264,0.5271,0.01,0.010,5
31,0.062694,0.5267,0.01,0.010,15
34,0.063565,0.5260,0.01,0.001,15
32,0.058907,0.5257,0.01,0.010,25


In [71]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.01, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform4 = Nystroem(kernel = 'rbf',random_state = 0,gamma=0.001,n_components=5)

clf4 = make_pipeline(transform4,model1)

# Entraînement
classifier4 = delayed(clf4.fit)(X_train_scale)

# Prédiction
y_pred4 = delayed(classifier4.predict)(X_test_scale).compute()

CPU times: total: 11 s
Wall time: 8.18 s


In [72]:
%%time
resultats(y_pred4)

Confusion matrix:
 [[   411   6162]
 [  5974 734863]]
F1_score :
 0.06343571538817717
Estimateurs :
               precision    recall  f1-score   support

          -1       0.06      0.06      0.06      6573
           1       0.99      0.99      0.99    740837

    accuracy                           0.98    747410
   macro avg       0.53      0.53      0.53    747410
weighted avg       0.98      0.98      0.98    747410

Auc Score :
 0.5272
Marge = 3093333.17 euros
CPU times: total: 4.58 s
Wall time: 3.9 s


Nous pouvons constater que les valeurs faibles de Gamma sont à privilégier. Elles indiquent un grand rayon de similarité, ce qui entraîne le regroupement de plus de points. Ainsi que le nombre de composants.

In [73]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.01, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform5 = Nystroem(kernel = 'poly',random_state = 0,degree=5,gamma=0.001,n_components=5)

clf5 = make_pipeline(transform5,model1)

# Entraînement
classifier5 = delayed(clf5.fit)(X_train_scale)

# Prédiction
y_pred5 = delayed(classifier5.predict)(X_test_scale).compute()

CPU times: total: 11.9 s
Wall time: 8.51 s


In [77]:
%%time
resultats(y_pred5)

Confusion matrix:
 [[   260   6313]
 [  3663 737174]]
F1_score :
 0.04954268292682927
Estimateurs :
               precision    recall  f1-score   support

          -1       0.07      0.04      0.05      6573
           1       0.99      1.00      0.99    740837

    accuracy                           0.99    747410
   macro avg       0.53      0.52      0.52    747410
weighted avg       0.98      0.99      0.98    747410

Auc Score :
 0.5173
Marge = 2189971.95 euros
CPU times: total: 4.19 s
Wall time: 3.88 s


In [75]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.01, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform6 = Nystroem(kernel = 'sigmoid',random_state = 0,gamma=0.001,n_components=5)

clf6 = make_pipeline(transform6,model1)

# Entraînement
classifier6 = delayed(clf6.fit)(X_train_scale)

# Prédiction
y_pred6 = delayed(classifier6.predict)(X_test_scale).compute()

CPU times: total: 10.7 s
Wall time: 7.93 s


In [78]:
%%time
resultats(y_pred6)

Confusion matrix:
 [[   245   6328]
 [  3299 737538]]
F1_score :
 0.04843333003854898
Estimateurs :
               precision    recall  f1-score   support

          -1       0.07      0.04      0.05      6573
           1       0.99      1.00      0.99    740837

    accuracy                           0.99    747410
   macro avg       0.53      0.52      0.52    747410
weighted avg       0.98      0.99      0.99    747410

Auc Score :
 0.5164
Marge = 2027618.47 euros
CPU times: total: 4.11 s
Wall time: 3.86 s


## 3.3 Amélioration des performances avec stratégie de sur-échantillonnage

In [90]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.01, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform4 = Nystroem(kernel = 'rbf',random_state = 0,gamma=0.001,n_components=5)

clf4 = make_pipeline(transform4,model1)

# Entraînement avec BorderlineSMOTE
classifier7 = delayed(clf4.fit)(X_train_border_scale)

# Prédiction
y_pred7 = delayed(classifier7.predict)(X_test_scale).compute()

CPU times: total: 19.8 s
Wall time: 15.9 s


In [91]:
%%time
resultats(y_pred7)

Confusion matrix:
 [[   261   6312]
 [  3300 737537]]
F1_score :
 0.05150976909413854
Estimateurs :
               precision    recall  f1-score   support

          -1       0.07      0.04      0.05      6573
           1       0.99      1.00      0.99    740837

    accuracy                           0.99    747410
   macro avg       0.53      0.52      0.52    747410
weighted avg       0.98      0.99      0.99    747410

Auc Score :
 0.5176
Marge = 2620255.41 euros
CPU times: total: 4.64 s
Wall time: 3.78 s


In [108]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.01, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform4 = Nystroem(kernel = 'rbf',random_state = 0,gamma=0.001,n_components=5)

clf4 = make_pipeline(transform4,model1)

# Entraînement avec ADASYN
classifier8 = delayed(clf4.fit)(X_train_ada_scale)

# Prédiction
y_pred8 = delayed(classifier8.predict)(X_test_scale).compute()

CPU times: total: 21.9 s
Wall time: 17.4 s


In [109]:
%%time
resultats(y_pred8)

Confusion matrix:
 [[   279   6294]
 [  3495 737342]]
F1_score :
 0.053928674978254564
Estimateurs :
               precision    recall  f1-score   support

          -1       0.07      0.04      0.05      6573
           1       0.99      1.00      0.99    740837

    accuracy                           0.99    747410
   macro avg       0.53      0.52      0.52    747410
weighted avg       0.98      0.99      0.99    747410

Auc Score :
 0.5189
Marge = 2614618.52 euros
CPU times: total: 4.14 s
Wall time: 3.86 s


## 3.4 Stratégie de post-traitement avec focalisation sur les individus de la classe majoritaire

In [105]:
df_train_maj = df_train[df_train.FlagImpaye == 1]

In [106]:
y_train_maj = df_train_maj["FlagImpaye"]
# Définir les variables explicatives 
X_train_maj = df_train_maj.drop(["FlagImpaye"], axis = "columns")

In [111]:
X_train_maj_scale = scaler.fit_transform(X_train_maj)

In [112]:
%%time
model1 = linear_model.SGDOneClassSVM(nu = 0.01, random_state= 0, tol=1e-7, max_iter = 2000)

# Kernel approximation

transform4 = Nystroem(kernel = 'rbf',random_state = 0,gamma=0.001,n_components=5)

clf4 = make_pipeline(transform4,model1)

# Entraînement
classifier9 = delayed(clf4.fit)(X_train_maj_scale)

# Prédiction
y_pred9 = delayed(classifier9.predict)(X_test_scale).compute()

CPU times: total: 2.8 s
Wall time: 1.52 s


In [113]:
%%time
resultats(y_pred9)

Confusion matrix:
 [[   267   6306]
 [  3623 737214]]
F1_score :
 0.05103698747969035
Estimateurs :
               precision    recall  f1-score   support

          -1       0.07      0.04      0.05      6573
           1       0.99      1.00      0.99    740837

    accuracy                           0.99    747410
   macro avg       0.53      0.52      0.52    747410
weighted avg       0.98      0.99      0.99    747410

Auc Score :
 0.5179
Marge = 2716821.52 euros
CPU times: total: 4.08 s
Wall time: 3.84 s
